In [1]:
%%capture

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?
import warnings
warnings.filterwarnings('ignore')
import calitp_data_analysis.magics

import pandas as pd
import numpy as np
import geopandas as gpd
from siuba import *
import json

import shared_utils
import warnings
from path_example_vars import GCS_PATH

import conveyal_path_utils
import branca

In [2]:
from shapely.ops import split, substring, LineString
from calitp_data_analysis import geography_utils
from IPython.display import Markdown

In [3]:
ratio_cmap = branca.colormap.step.Spectral_05
ratio_cmap.colors.reverse() #  note this modifies inplace
ratio_cmap = ratio_cmap.scale(vmin=1, vmax=3)
ratio_cmap.caption = 'Transit/Auto Travel Time Ratio'

In [4]:
xfer_cmap = branca.colormap.step.Spectral_04
xfer_cmap.colors.reverse() #  note this modifies inplace
xfer_cmap = xfer_cmap.scale(vmin=0, vmax=4)
xfer_cmap.caption = 'Number of Transfers'

In [5]:
%%capture_parameters
region = 'solano'
region

In [6]:
regions = ['la', 'sac', 'clovis', 'napa', 'solano']
assert region in regions
region_human = ['Los Angeles', 'Sacramento', 'Clovis', 'Napa', 'Solano']
region_human = dict(zip(regions, region_human))

In [9]:
# display(Markdown(f'# {region_human[region]}'))

# {region}

In [8]:
auto_df = pd.read_parquet(f'{GCS_PATH}streetlight_results.parquet')

In [ ]:
# auto_df.city = auto_df.city.str.replace('Solano', 'Sonoma')
auto_df.set_index('city', drop=True, inplace=True)

In [ ]:
car_p50_time = auto_df.loc[region_human[region], '50_ttp_minutes']
if region == 'clovis':
    car_p50_time = car_p50_time * 0.7 #  scale since transit route is shorter than planned

In [ ]:
# car_p50_time

In [ ]:
df = conveyal_path_utils.read_conveyal_path_df(f'{GCS_PATH}{region}_PATHS.csv')
df = conveyal_path_utils.add_warehouse_identifiers(df)

In [ ]:
points = pd.read_csv(f'{GCS_PATH}{region}_points.csv')
points = gpd.GeoDataFrame(points, geometry=gpd.points_from_xy(points.lon, points.lat)
                          , crs=geography_utils.WGS84).to_crs(geography_utils.CA_NAD83Albers)
points.geometry = points.buffer(100)

In [ ]:
#  get warehouse data
warehouse_data = conveyal_path_utils.get_warehouse_data(df)
spatial_routes = conveyal_path_utils.compile_all_spatial_routes(df, warehouse_data, verbose=False)

In [ ]:
def map_trip_groups(spatial_routes: pd.DataFrame, points: gpd.GeoDataFrame, which='trip_group_id'):
    
    col_list = ['trip_group_id', 'xfer_count', 'car_p50_ratio',
                     'route_name']
    cmaps = ['tab20', xfer_cmap, ratio_cmap, 'tab20']
    cmaps = dict(zip(col_list, cmaps))
    # display(cmaps)
    assert which in col_list
    display_list = ['optimal_pct', 'total_time', 'route_optimal_pct']
    human_names = {col: col.replace('_', ' ').title() for col in col_list + display_list}
    human_names['name'] = 'GTFS Feed Name'
    map_routes = spatial_routes.copy() >> select(-_.trip_id, -_.route_short_name, -_.route_long_name,
                                                         -_.stop_id, -_.stop_sequence, -_.stop_geom)
    map_routes['car_p50_ratio'] = map_routes.total_time / car_p50_time
    if which != 'route_name':
        map_routes.segment_geom = map_routes.apply(lambda x: x.segment_geom.buffer(min(x.optimal_pct * 800, 500)), axis=1)
        map_routes = map_routes >> arrange(-_.optimal_pct)
    else:
        route_grouped = (spatial_routes >> group_by(_.route_name, _.segment_geom, _.name)
                            >> summarize(route_optimal_pct = _.optimal_pct.sum())
                        )
        route_grouped.segment_geom = route_grouped.apply(lambda x: x.segment_geom.buffer(min(x.route_optimal_pct * 800, 500)), axis=1)
        route_grouped = route_grouped >> arrange(-_.route_optimal_pct)
        map_routes = gpd.GeoDataFrame(route_grouped, geometry='segment_geom', crs=geography_utils.CA_NAD83Albers).round(2)

    map_routes = map_routes.rename(columns=human_names).round(2).fillna('none') # needed for explore to work?
    # return(map_routes)
    m = map_routes.explore(column = human_names[which], cmap=cmaps[which], tiles="CartoDB positron")
    points = points.replace({0:'Origin', 1:'Destination'})
    points = points >> arrange(_.od)
    points.rename(columns={'od':'Origin and Destination'}, inplace=True)
    points_style = {'fillOpacity': 1}
    return points.explore(column='Origin and Destination', cmap=['#d95f02', '#1b9e77'], m=m, style_kwds=points_style, legend=False)

## Trip Group Map

This map shows each "trip group": a routing that was optimal for at least part of the time during the analysis window. Line width is based on the percent of the overall time for which that routing was optimal.

In all maps, origins are in green and destinations in orange.

In [ ]:
map_trip_groups(spatial_routes, points, 'trip_group_id')

## Auto Travel Time Comparison Map

For each trip group, this map shows the ratio between the best-case transit travel time and the median car travel time (based on Streetlight data including congested conditions).

In [ ]:
map_trip_groups(spatial_routes, points, 'car_p50_ratio')

## Route-aggregated Map

This map does not map individual trip groups, but instead aggregates all trip groups using a particular transit route. It shows which routes are most often used across all trip groups.

In [ ]:
map_trip_groups(spatial_routes, points, 'route_name')

## Transfer Count Map

For each trip group, this map shows the number of transfers.

In [ ]:
map_trip_groups(spatial_routes, points, 'xfer_count')

In [ ]:
la_md = '''
### LA: Regional Rail provides few transfers and fast trips, but isn't frequent enough to be available

* only two usable trips for 8-10am departures from origin, at 10:41 (Metrolink) and 11:01 (Amtrak)
'''

napa_md = '''
### Napa: Amtrak Thruway bus is best trip, but actually only departs once

* only one trip from Napa to Vallejo at 9:30!
* shows a high availability percent since even waiting 20-30min for the thruway bus is faster than the next-best option
'''

sac_md = '''
### Sacramento: most available routing involves 20min walk, isn't competitive with car
'''

clovis_md = '''
### Clovis: Only one possible routing, which isn't very direct

* pending network redesign will improve things
'''

vallejo_md = '''
### Vallejo: One way around the loop is faster once onboard, but wait times mean the optimal trip sometimes goes the long way 'round
'''

In [ ]:
which_md = {'la': la_md, 'napa': napa_md, 'sac': sac_md, 'clovis': clovis_md, 'solano': vallejo_md}

In [ ]:
display(Markdown(which_md[region]))